In [2]:
import os
import json
import PIL.Image
import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision

from src.data.PascalVOC import dataset_voc, prepare_dataloaders_pascal_voc
from src.attacks.attacks import FastGradientSign, ProjectedGradientDescent, UniversalAttack
from src.training.Trainer import Trainer
from src.optim.scheduler import CustomScheduler


def seed_everything(seed_value=4995):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def load_model(num_classes = 20, model_path = None, to_cuda = True):
    if not model_path:
        model = torchvision.models.resnet18(pretrained = True)
        input_feat = model.fc.in_features
        
        model.fc = nn.Linear(input_feat, num_classes)
        model.fc.reset_parameters()
        loaded_state_dict = False
    
    else:
        print("Loaded", model_path)
        model = torchvision.models.resnet18()
        input_feat = model.fc.in_features
        model.fc = nn.Linear(input_feat, num_classes)
        loaded_model = torch.load(model_path)
        model.load_state_dict(loaded_model['model_state_dict'])
        loaded_state_dict = True
        
    if to_cuda:
        model = model.to('cuda')
        
    return model, loaded_state_dict




In [4]:
seed_everything()
root_dir = 'Data/PascalVOC/VOCdevkit/VOC2012'

config = {
    'lr': 0.0001,
    'batch_size': 64,
    'weight_decay': 0.01
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = 'models/pascal_voc_resnet_fine_tuned.pt'

# Fine-tune all layers
resnet, loaded_state_dict = load_model(model_path = model_path)

for name, param in resnet.named_parameters():
    param.requires_grad = True

dataloaders, classes = prepare_dataloaders_pascal_voc(root_dir, batch_size = config['batch_size'], smaller_resize = True)

loss_fn = nn.BCEWithLogitsLoss(weight=None, reduction='mean')

train_params = [p for p in resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = config['lr'], weight_decay = config['weight_decay'])
#scheduler = CustomScheduler(optimizer)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4,6], gamma=0.1)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}



vec = torch.load('models/uni_pert_pascal_voc.pt')
attack = UniversalAttack(vec)


trainer = Trainer(
    model = resnet,
    loss_fn = loss_fn,
    classes = classes,
    num_epochs = 10,
    model_name = f'pascal_voc_uni_adv_trained',
    training_params = training_params,
    multi_label = True,
    print_frequency = 10,
    adversarial_training = True,
    adversarial_attack = attack,
    custom_scheduler = False
)

trainer.train()


print("Losses:",trainer_fine_tune.epoch_loss)
print("Accuracies:",trainer_fine_tune.epoch_mape)

Loaded models/pascal_voc_resnet_fine_tuned.pt
Current Epoch: 0
Train Loop:
Batch: 0 of 90. Loss: 0.2021379917860031. Mean so far: 0.2021379917860031. Mean of 100: 0.2021379917860031
Batch: 10 of 90. Loss: 0.16344387829303741. Mean so far: 0.16806463626298038. Mean of 100: 0.16806463626298038
Batch: 20 of 90. Loss: 0.1381864994764328. Mean so far: 0.15990073340279715. Mean of 100: 0.15990073340279715
Batch: 30 of 90. Loss: 0.15097014605998993. Mean so far: 0.1568525955080986. Mean of 100: 0.1568525955080986
Batch: 40 of 90. Loss: 0.1724778711795807. Mean so far: 0.1563943608141527. Mean of 100: 0.1563943608141527
Batch: 50 of 90. Loss: 0.14087842404842377. Mean so far: 0.1534918800872915. Mean of 100: 0.1534918800872915
Batch: 60 of 90. Loss: 0.1366935819387436. Mean so far: 0.15174675818349495. Mean of 100: 0.15174675818349495
Batch: 70 of 90. Loss: 0.1569652408361435. Mean so far: 0.15102839994598444. Mean of 100: 0.15102839994598444
Batch: 80 of 90. Loss: 0.1324392408132553. Mean so 

NameError: name 'trainer_fine_tune' is not defined